In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import torch
import importlib
module = importlib.import_module('transformers.models.qwen2.modeling_qwen2')
from transformers import AutoConfig, AutoModelForCausalLM
import time
from transformers.models.qwen2.modeling_qwen2 import *
from transformers.models.llama.modeling_llama import *
from copy import deepcopy
from tqdm import tqdm
import transformers

/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [2]:

# config = AutoConfig.from_pretrained('/mnt/workspace/mdy/models/Qwen2.5-7B-Instruct')
dtype = torch.bfloat16
bs = 4
seq_len = 2048
iters = 50
model_path = '/mnt/workspace/mdy/models/Qwen2.5-0.5B'
model_path = '/mnt/workspace/mdy/models/Llama-3.2-3B-Instruct'
# model_path = '/mnt/workspace/mdy/models/Meta-Llama-3.1-8B-Instruct'
old = AutoModelForCausalLM.from_pretrained(model_path,
                                            _attn_implementation='flash_attention_2',
                                           device_map='cuda', torch_dtype=dtype)
input_ids = torch.randint(10000, 20000, (bs, seq_len)).cuda()
pos_ids = torch.arange(seq_len)[None, :].expand(bs, -1).contiguous().cuda()
labels = input_ids
grad_list = []
for p in old.parameters():
    grad_list.append(p)
def zero_gard():
    for p in grad_list:
        if p.grad is not None:
            p.grad.zero_()


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.61s/it]


In [11]:
torch.cuda.empty_cache()
t = time.time()
for i in tqdm(range(iters)):
    out = old(input_ids, position_ids=pos_ids, labels=labels)
    out.loss.backward()
    break
t1 = time.time() - t
t1

  0%|          | 0/10 [00:00<?, ?it/s]


0.10018134117126465

In [12]:
print(grad_list[10].grad[:5])
zero_gard()

tensor([[ 5.2643e-04, -8.2016e-04, -2.5787e-03,  ...,  1.3580e-03,
         -9.9945e-04, -6.1798e-04],
        [-1.7452e-04,  5.0735e-04,  7.8583e-04,  ..., -2.2173e-05,
         -4.0245e-04,  8.0109e-04],
        [ 1.1292e-03, -1.4114e-03, -1.8616e-03,  ..., -5.9891e-04,
         -2.5787e-03, -1.6212e-05],
        [-1.0300e-03,  1.3504e-03, -3.5858e-03,  ...,  5.4932e-04,
          4.8828e-04, -1.8311e-03],
        [-7.5531e-04,  2.1210e-03, -2.6703e-04,  ...,  1.2817e-03,
          2.0752e-03, -8.0490e-04]], device='cuda:0', dtype=torch.bfloat16)


In [ ]:
from replace_qwen2_kernel import trigger
from replace_llama3_kernel import trigger

In [4]:
torch.cuda.empty_cache()
t = time.time()
for i in tqdm(range(iters)):
    out = old(input_ids, position_ids=pos_ids,labels=labels)
    out.loss.backward()
t2 = time.time() - t
t2

100%|██████████| 50/50 [00:43<00:00,  1.15it/s]


43.50182127952576

In [21]:
print(grad_list[10].grad[:5])
zero_gard()

tensor([[ 5.9509e-04, -7.9727e-04, -2.5482e-03,  ...,  1.2894e-03,
         -8.8882e-04, -6.9809e-04],
        [-3.4332e-04,  4.6349e-04,  8.1635e-04,  ..., -1.3351e-04,
         -4.3869e-04,  8.0109e-04],
        [ 1.0834e-03, -1.4267e-03, -1.9531e-03,  ..., -7.4387e-04,
         -2.5787e-03, -4.2200e-05],
        [-9.8419e-04,  1.4267e-03, -3.6621e-03,  ...,  4.8828e-04,
          5.1117e-04, -2.0752e-03],
        [-8.0109e-04,  2.1820e-03, -3.7384e-04,  ...,  1.2207e-03,
          2.0752e-03, -7.5531e-04]], device='cuda:0', dtype=torch.bfloat16)


In [9]:
print(grad_list[10].grad[:5])
zero_gard()

tensor([[ 3.5477e-04, -4.2343e-04, -2.0752e-03,  ..., -2.5558e-04,
         -6.7711e-05,  1.2779e-04],
        [-7.7820e-04, -7.3624e-04, -3.2043e-03,  ...,  5.3406e-04,
         -1.3733e-04,  2.1057e-03],
        [-6.3705e-04,  1.4365e-05, -1.8616e-03,  ..., -3.9482e-04,
          5.8746e-04, -8.5449e-04],
        [ 4.9591e-04,  4.6921e-04, -3.6621e-04,  ..., -1.9455e-04,
          2.0027e-04, -2.1815e-05],
        [-3.8719e-04, -3.8624e-05, -8.6975e-04,  ..., -1.3504e-03,
          3.6240e-04, -9.9945e-04]], device='cuda:0', dtype=torch.bfloat16)


In [ ]:
repeat_kv()

tensor([[-1.1953, -0.4570, -0.5195,  ..., -0.8477,  0.4590, -0.1875],
        [-0.1128,  0.5000,  0.6367,  ...,  1.4531, -1.0469, -1.4531],
        [-0.4531,  0.7734, -0.5117,  ...,  0.1680,  0.0503, -0.7305],
        ...,
        [-0.2012,  0.1729,  1.1719,  ..., -1.0234, -0.8516, -1.0234],
        [ 0.6406,  0.0623, -0.0825,  ...,  0.5117,  0.1953,  0.2637],
        [-0.5742, -0.4512,  1.4219,  ..., -2.5938,  0.4336, -0.4824]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [13]:
old_grad[0]

tensor([[ 1.2578, -0.6133,  0.4766,  ...,  0.2285, -0.2812, -2.7969],
        [ 1.2344, -1.3203,  0.8867,  ...,  0.0474, -0.5078,  0.2852],
        [-1.7734, -0.6367,  1.3516,  ...,  0.7344,  0.1299,  2.0000],
        ...,
        [ 0.1465,  0.7031,  0.2988,  ..., -1.1484,  0.8594, -1.3750],
        [-0.7930, -1.7812, -0.7969,  ..., -0.2637,  0.8594,  0.6445],
        [ 2.1250,  0.6328, -0.7852,  ...,  0.0801,  1.8281,  1.2500]],
       device='cuda:0', dtype=torch.bfloat16)

In [14]:
new_grad[0]

tensor([[ 1.2578, -0.6055,  0.4688,  ...,  0.2256, -0.2852, -2.8125],
        [ 1.2422, -1.3203,  0.8750,  ...,  0.0505, -0.5039,  0.3008],
        [-1.7734, -0.6211,  1.3594,  ...,  0.7227,  0.1260,  1.9922],
        ...,
        [ 0.1592,  0.6992,  0.2891,  ..., -1.1484,  0.8555, -1.3828],
        [-0.7969, -1.7812, -0.8086,  ..., -0.2578,  0.8750,  0.6523],
        [ 2.1406,  0.6211, -0.7773,  ...,  0.0732,  1.8281,  1.2578]],
       device='cuda:0', dtype=torch.bfloat16)